In [ ]:
from src.data.datasets.nuscenes import NuScenesValidationMini, LABEL_ID2NAME
import tqdm

In [ ]:
nusc_base_kwargs = {
    "patch_height": 256,
    "patch_aspect_ratio": 1.3,
    "label_names": ['car'],
    "data_root": "data/nuscenes",
    "pipeline": [],
    "box_type_3d": "Camera",
    "load_type": 'frame_based',
    "modality": dict(use_camera=True,use_lidar=False),
    "filter_empty_gt": False,
    "test_mode": False,
    "with_velocity": False,
    "use_valid_flag": False,
}

nusc_val = NuScenesValidationMini(**nusc_base_kwargs)

In [ ]:
len(nusc_val)

In [ ]:
idx = 0
nusc_val[idx].cam_instances[0]

In [ ]:
num_car_instances = 0
num_instances = 0
pbar = tqdm.tqdm(total=len(nusc_val))
instance_counts = {k: 0 for k in LABEL_ID2NAME.keys()}
for sample_idx, sample in enumerate(nusc_val):
    
    for cam_instance in sample.cam_instances:
        instance_counts[cam_instance.bbox_label] += 1
        
        pbar.set_description(f"Car instances: {num_car_instances}, Total instances: {num_instances}, Current instance: {cam_instance.bbox_label}")
        num_instances += 1
        if cam_instance.bbox_label == 0:
            num_car_instances += 1
    pbar.update(1)
print(instance_counts)


In [ ]:
instance_counts

In [ ]:
class AverageMeter:
    def __init__(self):
        self.reset()
        
    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0
        self.history = []
        
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count
        self.history.append(val)
        
    def __str__(self):
        return f"{self.avg:.4f}"
    
    def __float__(self):
        return self.avg

In [ ]:
# compute  patch size statistics - mean mode median min max
patch_size_meter_x = AverageMeter()
patch_size_meter_y = AverageMeter()

pbar = tqdm.tqdm(nusc_val)
for sample in nusc_val:
    for cam_instance in sample.cam_instances:
        patch_size_meter_x.update(cam_instance.patch.shape[1])
        patch_size_meter_y.update(cam_instance.patch.shape[0])

        pbar.set_description(f"Patch size x: {patch_size_meter_x}, Patch size y: {patch_size_meter_y}")
        pbar.update(1)
        



In [ ]:
print(f"Mean patch size: {patch_size_meter_x}, {patch_size_meter_y}")
# print median, mode, min, max

import numpy as np
import scipy.stats as stats
print(f"Mean patch size x: {np.mean(patch_size_meter_x.history)}, y: {np.mean(patch_size_meter_y.history)}")
print(f"Median patch size x: {np.median(patch_size_meter_x.history)}, y: {np.median(patch_size_meter_y.history)}")
print(f"Mode patch size x: {stats.mode(patch_size_meter_x.history)}, y: {stats.mode(patch_size_meter_y.history)}")
print(f"Min patch size x: {np.min(patch_size_meter_x.history)}, y: {np.min(patch_size_meter_y.history)}")
print(f"Max patch size x: {np.max(patch_size_meter_x.history)}, y: {np.max(patch_size_meter_y.history)}")
print(f"Patch size x std: {np.std(patch_size_meter_x.history)}, y: {np.std(patch_size_meter_y.history)}")
print(f"Patch size x var: {np.var(patch_size_meter_x.history)}, y: {np.var(patch_size_meter_y.history)}")

In [ ]:
# histogram of patch sizes
import matplotlib.pyplot as plt
plt.hist(patch_size_meter_x.history, bins=100)
plt.show()
plt.hist(patch_size_meter_y.history, bins=100)
plt.show()


In [ ]:
# aspect ratio statistics
aspect_ratio_meter = AverageMeter()
pbar = tqdm.tqdm(nusc_val)
for sample in nusc_val:
    for cam_instance in sample.cam_instances:
        aspect_ratio_meter.update(cam_instance.patch.shape[1] / cam_instance.patch.shape[0])
        pbar.set_description(f"Aspect ratio: {aspect_ratio_meter}")
        pbar.update(1)
        
print(f"Mean aspect ratio: {aspect_ratio_meter}")
print(f"Median aspect ratio: {np.median(aspect_ratio_meter.history)}")
print(f"Mode aspect ratio: {stats.mode(aspect_ratio_meter.history)}")
print(f"Min aspect ratio: {np.min(aspect_ratio_meter.history)}")
print(f"Max aspect ratio: {np.max(aspect_ratio_meter.history)}")
print(f"Aspect ratio std: {np.std(aspect_ratio_meter.history)}")
print(f"Aspect ratio var: {np.var(aspect_ratio_meter.history)}")

# histogram of aspect ratios
import matplotlib.pyplot as plt
plt.hist(aspect_ratio_meter.history, bins=100)
plt.show()


In [ ]:
# histogram of patch sizes
import matplotlib.pyplot as plt
plt.hist(patch_size_meter_x.history, bins=100)
plt.show()
plt.hist(patch_size_meter_y.history, bins=100)

plt.show()

In [ ]:
# display 10 samples in the original size
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

pbar = tqdm.tqdm(nusc_val)
counter = 0
for i, sample in enumerate(pbar):
    for cam_instance_idx, cam_instance in enumerate(sample.cam_instances):
        pbar.set_description(f"Sample {i}, Camera {cam_instance_idx}, Label: {cam_instance.bbox_label}, Patch shape: {cam_instance.patch.shape}")
        if cam_instance.bbox_label == 0:
            plt.imshow(cam_instance.patch)
            plt.show()
            counter += 1
        if counter >= 10:
            break
        
